In [2]:
import re
import requests
import pandas as pd
from pyquery import PyQuery as pq
from collections import namedtuple

from IPython.core.display import display, HTML

pd.set_option('display.max_colwidth', -1)

In [ ]:
def parse_tr(tr):
    tds = [td.text_content().strip() for td in e.findall('td')][2:]
    tds.append('http://wg-gesucht.de/' + tr.attrib['adid'])
    return tds

In [ ]:
url_template = {'einzimmer':'http://www.wg-gesucht.de/1-zimmer-wohnungen-in-Hamburg.55.1.0.{page}.html?offer_filter=1&city_id=55&category=1&rent_type=2&rMax=750&img_only=1',
                'wohnung':'http://www.wg-gesucht.de/wohnungen-in-Hamburg.55.2.0.{page}.html?offer_filter=1&sort_column=0&city_id=55&category=2&rent_type=2&rMax=750&img_only=1'}

cookies = {'einzimmer':None, 'wohnung':None}

listing_types = ['einzimmer', 'wohnung']
einzimmer = ['price', 'area', 'borough', 'frei_ab', 'frei_bis', 'url']
wohnung = [ 'date', 'price', 'area', 'borough', 'frei_ab', 'frei_bis', 'url']
page_format = {'einzimmer':einzimmer, 'wohnung':wohnung}

In [ ]:
dfs = []
for wohnung_typ in listing_types:
    for i in range(5):
        url = url_template[wohnung_typ].format(**{'page':i})
        res = requests.get(url, cookies=cookies[wohnung_typ])
        cookies[wohnung_typ] = res.cookies
        
        d = pq(res.text)        
        trs = d('#table-compact-list tr').not_('.inlistTeaser')[2:]
        ids = [tr.attrib['adid'] for tr in trs]
        
        data = [{k:v for k, v in zip(page_format[wohnung_typ], parse_tr(e))} for e in trs]
        df = pd.DataFrame(data, columns=page_format[wohnung_typ])
        inactive = (df.frei_ab == 'aktuell') & (df.frei_bis == 'vermietet')
        df = df[~inactive]
        df['wohnung_typ'] = wohnung_typ
        dfs.append(df)
        if any(inactive): break
df = pd.concat(dfs, ignore_index=True)

In [ ]:
df = pd.concat(dfs, ignore_index=True)
df['price'] = df.price.apply(lambda x: re.sub("[^0-9]", "", x))
df['area'] = df.area.apply(lambda x: re.sub("[^0-9]", "", x))
df['price'] = pd.to_numeric(df.price)
df['area'] = pd.to_numeric(df.area)
df['price_area_ratio'] = df.price/df.area

In [ ]:
df

In [ ]:
df[df.frei_bis == '']